Created on Wed Nov 23 20:39:32 2022
@author: bukold@uni-hildesheim.de

This script searches the Twitter-API for users. It Script may take a while, because the API pauses repeatedly for 15 Minutes.
Good Luck, Q. Bukold

In [ ]:
import tweepy as tw
import pandas as pd
import json
from pandas.io.json import json_normalize

In [ ]:
bearer_handler = ""
auth = tw.OAuth2BearerHandler(bearer_handler)
api = tw.API(auth, wait_on_rate_limit=True)

#define input df (should have one column called "query")
input_df = 
query_input = pd.DataFrame(input_df["query"])

#prepare output-df
df = pd.DataFrame(list())
df.to_csv("output.csv")
output = "output.csv" #size of the .csv does not matter

In [ ]:

# function to convert _json to JSON
def jsonify_tweepy(tweepy_object):
    json_str = json.dumps(tweepy_object._json)
    return json.loads(json_str)

def JSON_to_df(query_results):
    #using function
    list = [jsonify_tweepy(tweepy_json)for tweepy_json in query_results]
    # Convert followers_list to a pandas dataframe
    df = json_normalize(list)
    # filter dataframe
    return(df)

#actual search, change count-variable for less results per query
def search_api(query):
    result = api.search_users(query, count=5, include_entities=False)
    return result

# combine lists into df
def combine_lists_into_df(list1, list2):
    #lists -> pandas df
    user_df = pd.concat(list1)
    query_df = pd.DataFrame(list2)
    query_df.columns = ['query']
    user_df = user_df.reset_index()
    query_df = query_df.reset_index()
    #combine df of querys and df of found users
    connected_df = pd.concat([query_df, user_df], axis=1)
    connected_df = connected_df.drop(connected_df.columns[2], axis=1)
    return connected_df

In [ ]:
user_df = pd.DataFrame(columns = ['query', 'screen_name', 'name', 'id', 'description', 'verified', 'followers_count'])
user_list = []
query_list = []
not_found = []
repetitions = 0

for index, row in query_input.iterrows():
    #print progress
    repetitions = repetitions + 1
    print("***",repetitions, "von", len(query_input),"***")

    #search user via API
    user = (search_api(row["query"])) #add column-name of input-csv
    user_df_type = JSON_to_df(user).filter(['screen_name', 'name', 'id', 'description', 'verified', 'followers_count'],  axis=1)
    
    #if not found, append official_name to not_found-list
    if user_df_type.empty:
        not_found.append(row["query"])
    
    #else append results to user_list
    else:
        user_list.append(user_df_type)
        #for every result add query to list, in order to later combine query and results in one df
        for i in range(len(user_df_type.index)):
            query_list.append(row['query'])

    #save all X repetitions, in case of shut down
    if repetitions % 500 == 0: #copy to output every 500 querys
        #combine lists and save to csv
        combine_lists_into_df(user_list, query_list).to_csv(output, encoding='utf-8', index=False)
        #print how many official_names where not found
        print(len(not_found))

connected_df = combine_lists_into_df(user_list, query_list)
connected_df = connected_df.merge(input_df, on='query', how='left')
connected_df.to_csv(output, encoding='utf-8', index=False) 

#print number of querys with no result
print("***", len(not_found), "names where not found","***")